In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import importlib
import os
import kagglehub
import ast

In [2]:
import sys
sys.path.append('scripts/')
import scraping, merge_goodreads, merge_cmu, merge_imdb
from scraping import *
from merge_goodreads import *
from merge_cmu import *
from merge_imdb import *

# Scraping data from wikipedia
We must first define the url that we will scrap data from. They will allow us to make a mapping between books and their film adaptation.

In [3]:
# URL of the Wikipedia page
url_0_C = "https://en.wikipedia.org/wiki/List_of_fiction_works_made_into_feature_films_(0%E2%80%939,_A%E2%80%93C)"
url_D_J = "https://en.wikipedia.org/wiki/List_of_fiction_works_made_into_feature_films_(D%E2%80%93J)"
url_K_R = "https://en.wikipedia.org/wiki/List_of_fiction_works_made_into_feature_films_(K%E2%80%93R)"
url_S_Z = "https://en.wikipedia.org/wiki/List_of_fiction_works_made_into_feature_films_(S%E2%80%93Z)"
url_short = "https://en.wikipedia.org/wiki/List_of_short_fiction_made_into_feature_films"
url_kids = "https://en.wikipedia.org/wiki/List_of_children%27s_books_made_into_feature_films"

urls = [url_0_C, url_D_J, url_K_R, url_S_Z, url_short, url_kids]

Then, we scrap and process data from these wikipedia pages.

In [4]:
# Launches the scrapping on every url selected
dataframes = []
for url in urls: 
    df = scrap_book_to_movie(url)
    clean_df = scrap_post_processing(df)
    dataframes.append(clean_df)

book_adaptations = pd.concat(dataframes).reset_index(drop=True)
book_adaptations = book_adaptations.drop_duplicates().reset_index(drop=True)
book_adaptations.head()

/Users/hugo/Documents/EPFL/MA3/ADA/homeworks/ada-2024-project-padamalgame/src/scripts/scraping.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['title_book'] = df['fiction_work'].str.split('(').str[0]
/Users/hugo/Documents/EPFL/MA3/ADA/homeworks/ada-2024-project-padamalgame/src/scripts/scraping.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['title_book'] = df['title_book'].apply(lambda t: t.replace('"', ''))
/Users/hugo/Documents/EPFL/MA3/ADA/homeworks/ada-2024-project-padamalgame/src/script

,title_book,author_book,year_book,title_film,year_film
0,The 25th Hour,David Benioff,2001,25th Hour,2002
1,3 Assassins,Kōtarō Isaka,2004,Grasshopper,2015
2,4.50 from Paddington,Agatha Christie,1957,"Murder, She Said",1961
3,4.50 from Paddington,Agatha Christie,1957,Crime Is Our Business,2008
4,58 Minutes,Walter Wager,1987,Die Hard 2,1990


We now have a dataframe with 4941 film adaptations together with the book they are adapting.

# Merge with Goodreads
We will now merge the book to movie mapping with the goodreads dataset to have additional information on the books.


First we download the dataset from kaggle

In [5]:
path = kagglehub.dataset_download("bahramjannesarr/goodreads-book-datasets-10m")

In [6]:
df_movies = book_adaptations.copy()
df_goodreads = books_csv_to_df(path)

df_goodreads['merge_authors'] = clean_spaces(df_goodreads['Authors'])
df_goodreads['merge_names'] = clean_spaces(df_goodreads['Name'])
df_goodreads['merge_names'] = remove_parenthesis(df_goodreads['merge_names'])


df_movies['merge_authors'] = clean_spaces(df_movies['author_book'])
df_movies['merge_names'] = clean_spaces(df_movies['title_book'])
df_movies['merge_names'] = remove_parenthesis(df_movies['merge_names'])

merge_goodreads = df_goodreads.merge(right=df_movies, how="right", left_on=['merge_authors', 'merge_names'], right_on=['merge_authors', 'merge_names'], copy=False)
merge_goodreads = merge_goodreads.drop_duplicates(subset = df_movies.columns).reset_index(drop=True)
merge_goodreads = merge_goodreads.drop(columns = ['merge_authors', 'merge_names', 'Authors', 'Name'])

# Merge with CMU
We will now merge this data with the CMU dataset to add extra information on these films.

In [ ]:
# Merge df with CMU depending on title_film and year_film
merge_cmu = merge_with_CMU(merge_goodreads)
merge_cmu.head()

Now we have more information on the films that are an adaptation of a book, such as their genres. Let's add more information such as the film's rating by merging with IMDB's dataset.

# Merge with IMDB


In [ ]:
merge_imdb = merge_with_imdb(merge_cmu)
merge_imdb.to_csv('merge_imdb.csv', index=False)
merge_imdb.head()

We now have 1940 film samples that are adaptations from known books and which can use for analysis.